In [ ]:

# Install dependencies (run this in your notebook environment)
!pip install fastapi uvicorn pymongo dnspython

In [ ]:

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from pymongo import MongoClient

# Initialize FastAPI app
app = FastAPI(title="Week 4 - FastAPI & MongoDB Demo")

# MongoDB Connection
client = MongoClient("mongodb://localhost:27017/")  # Replace with your MongoDB URI if remote
db = client["week4_db"]
collection = db["students"]

print("Connected to MongoDB successfully!")

In [ ]:

class Student(BaseModel):
    name: str
    age: int
    course: str

In [ ]:

@app.post("/students/")
def create_student(student: Student):
    data = student.dict()
    result = collection.insert_one(data)
    return {"message": "Student added successfully", "id": str(result.inserted_id)}


@app.get("/students/")
def get_students():
    students = list(collection.find({}, {"_id": 0}))
    return {"students": students}


@app.get("/students/{name}")
def get_student(name: str):
    student = collection.find_one({"name": name}, {"_id": 0})
    if not student:
        raise HTTPException(status_code=404, detail="Student not found")
    return student


@app.put("/students/{name}")
def update_student(name: str, student: Student):
    updated_data = student.dict()
    result = collection.update_one({"name": name}, {"$set": updated_data})
    if result.matched_count == 0:
        raise HTTPException(status_code=404, detail="Student not found")
    return {"message": "Student updated successfully"}


@app.delete("/students/{name}")
def delete_student(name: str):
    result = collection.delete_one({"name": name})
    if result.deleted_count == 0:
        raise HTTPException(status_code=404, detail="Student not found")
    return {"message": "Student deleted successfully"}

In [ ]:

# Run the server (use in terminal, not in notebook)
# uvicorn main:app --reload

# Example test endpoints:
# POST    /students/       -> Add a new student
# GET     /students/       -> Get all students
# GET     /students/{name} -> Get a specific student
# PUT     /students/{name} -> Update a student
# DELETE  /students/{name} -> Delete a student